IR Tool:

Pre-processes text
- Tokenisation
- Stopword removal using this list of words. You MUST use this list.
- Porter stemming. You can use packages for this part, such as Snowball or NLTK.

Creates a positional inverted index
    Your index can have whatever structure you like, and can be stored in any format you like, but you will need to output it to a text file using the format specfied below.


In [34]:
from nltk.stem.porter import PorterStemmer
import xml.dom.minidom  
import string
import re
import math

In [35]:
'''
changes the .xml to the change.txt in root,
only changes the contant of what I need: Document Number, Headlin and text.
'''
def changeXML(txt_path):    

    #The path to store trec.sample.xml, change it as needed 
    #e.g. txt_path='D://ATFC//TTDS//Sem1//CW1//collections//trec.sample.xml'
    Txt='change.txt'
    DOMTree = xml.dom.minidom.parse(txt_path)
    annotation = DOMTree.documentElement  
    objects = annotation.getElementsByTagName("DOC")  
    f = open(Txt, "w")

    for object in objects:  
        a1 = object.getElementsByTagName("DOCNO")[0]
        if a1.childNodes == []:
                b1 = ''
        else:
            b1 = a1.childNodes[0].data
    
        a2 = object.getElementsByTagName("HEADLINE")[0]
        if a2.childNodes == []:
            b2 = ''
        else:
            b2 = a2.childNodes[0].data
            
        a3 = object.getElementsByTagName("TEXT")[0]
        if a3.childNodes == []:
            b3 = ''
        else:
            b3 = a3.childNodes[0].data
              
        contents = 'ID: '+b1 +'\nHeadline '+ b2 + b3
        f.write(contents)
    f.close()

        
def tokenisation():
    stri = open('change.txt').read()
    for i in stri:
        if i in string.punctuation:
            stri = stri.replace(i," ")
    with open('change.txt','w') as f:
        f.write("{}\n".format(stri))
    f.close()

    
def stopwordANDstemming(stop_word_path):
    #The path to store englishST.txt, change it as needed  
    porter_stemmer = PorterStemmer()
    afterstri = []
    global stop_words
    global word_list
    sto = open(stop_word_path).read()
    stop_words = set(sto)
    
    with open('change.txt','r') as f :
        lines = f.readlines()
        for line in lines:
            lline = line.split()
            for i in lline:
                if i not in stop_words:
                    i = porter_stemmer.stem(i)
                    afterstri.append(i)
    f.close()
    
    with open('change.txt','w') as fs:
        for i in afterstri:
            if i =='id':
                fs.write('\n')
            if i == 'headlin':
                fs.write(" ")
            else:
                fs.write(" {}".format(i))
    fs.close()
    
    word_list = list(set(afterstri))
    word_list.sort()
             

In [36]:
def index():
    
    global word_list
    global dictionary
    context = []
    docno = []
    global Word_Documentno
    global worddf

    with open('change.txt','r') as fi:
        lines = fi.readlines()
        for line in lines:
            lline=line.split()
            if not len(lline):
                continue
            else:
                docno.append(lline[1])
                context.append(lline[2:])
    fi.close()
    dictionary = dict(zip(docno,context))
    
    for word in word_list:
        df = 0
        memol = []
        for key in dictionary.keys():
            if word in dictionary[key]:
                memol.append(key)
                df = df+1
        Word_Documentno.append(memol)
        worddf.append(df)

    a = 0
    with open('index.txt','w') as f:
        for word in word_list:   
            f.write("{0}:({1})\n".format(word, worddf[a]))
            line = Word_Documentno[a]
            for number in line:
                f.write("\t\t{0}:{1}\n".format(number,[i for i,x in enumerate(dictionary[number]) if x ==word]))
            a = a+1
    f.close()   
   


Uses your positional inverted index to perform:
- Boolean search
- Phrase search
- Proximity search
- Ranked IR based on TFIDF

In [37]:
def Search(query):
    
    global word_list
    global stop_words
    global Word_Documentno
    porter_stemmer = PorterStemmer()

    if query not in stop_words:
        query_use= porter_stemmer.stem(query)
        documents = Word_Documentno[word_list.index(query_use)]
        flag = 1
        score = [0]
    else:
        flag = 1
        documents = 'This word is a stop word!'
        score = [0]
    return flag, documents, score

    
    
def PhraseSearch(query, prox):
    
    global word_list
    global Word_Documentno
    global dictionary
    
    porter_stemmer = PorterStemmer()
    documentS = []
    documentSs_together = []
    documentsS1 = []
    documentsS2 = []
    
    punctuation_string = string.punctuation
    for i in query:
        if i in string.punctuation:
            query = query.replace(i," ")
    Q = query.split()
    word1 = Q[0]
    word2 = Q[1]
    
    word1_use= porter_stemmer.stem(word1)
    documentsS1 = Word_Documentno[word_list.index(word1_use)]
        
    word2_use= porter_stemmer.stem(word2)
    documentsS2 = Word_Documentno[word_list.index(word2_use)]
    documentSs_together = list(set(documentsS1)|set(documentsS2))
    
    if len(documentSs_together)<1:
        documentS = ['There no document include this phrase.']
    else:
        for number in documentSs_together:
            list1 = [i for i,x in enumerate(dictionary[number]) if x ==word1_use]
            list2 = [y for y,x in enumerate(dictionary[number]) if x ==word2_use]
            for i in list1:
                for y in list2:
                    if abs(i-y)<=prox:
                        documentS.append(number)
    flag = 1
    score = [0]
    documentS = list(set(documentS))
    documentS = [int(x) for x in documentS]
    documentS.sort()
    documentS = [str(x) for x in documentS]

    return flag, documentS, score
                    
             
    
def BooleanSearch(word1, operator, word2):
    
    global word_list
    global Word_Documentno
    porter_stemmer = PorterStemmer()
    documents_together = []
    documents1 = []
    documents2 = []
    
    if '"' in word1:
        flag1, documents1, score1 = PhraseSearch(word1, prox = 1)
    else:
        word1_use= porter_stemmer.stem(word1)
        documents1 = Word_Documentno[word_list.index(word1_use)]
        
    if  '"' in word2:
        flag2, documents2, score2 = PhraseSearch(word2, prox = 1)
    else:
        word2_use= porter_stemmer.stem(word2)
        documents2 = Word_Documentno[word_list.index(word2_use)]
        
    if operator == 'AND':
        documents_together = list(set(documents1)&set(documents2))
    elif operator == 'OR':
        documents_together = list(set(documents1)|set(documents2))
    elif operator == 'AND NOT':
        documents_together = list(set(documents1)-set(documents2))
    else:
        documents_together[0] = 'Ohoh...The operator has something wrong...'
        
    flag = 1
    score = [0]
    documents_together = list(set(documents_together))
    documents_together = [int(x) for x in documents_together]
    documents_together.sort()
    documents_together = [str(x) for x in documents_together]

    return flag, documents_together, score
            
               
    
def get_proxomity_query_elements(query):  
    
    sign_open = query.index('(')
    sign_close = query.index(')')
    proximity_number = int(query[1:sign_open])
    q = query[sign_open + 1:sign_close]
    
    punctuation_string = string.punctuation
    for i in q:
        if i in string.punctuation:
            q = q.replace(i," ")
    Q = q.split()
    word1 = Q[0]
    word2 = Q[1]
        
    return word1, word2, proximity_number
    

    
def Proximitysearch(word1, word2, proximity_number):
    global word_list
    global Word_Documentno
    porter_stemmer = PorterStemmer()
    documentP = []
    documentsP1 = []
    documentsP2 = []
    documents_togetherP = []
    
    word1_use= porter_stemmer.stem(word1)
    documentsP1 = Word_Documentno[word_list.index(word1_use)]
        
    word2_use= porter_stemmer.stem(word2)
    documentsP2 = Word_Documentno[word_list.index(word2_use)]
    documents_togetherP = list(set(documentsP1)|set(documentsP2))
    
    if len(documents_togetherP)<1:
        documentP[0] = ['There no document include this phrase.']
    else:
        for number in documents_togetherP:
            list1 = [i for i,x in enumerate(dictionary[number]) if x ==word1_use]
            list2 = [y for y,x in enumerate(dictionary[number]) if x ==word2_use]
            for i in list1:
                for y in list2:
                    if abs(i-y)<=proximity_number:
                        documentP.append(number)
    
    flag = 1
    score = [0]
    documentP = list(set(documentP))
    documentP = [int(x) for x in documentP]
    documentP.sort()
    documentP = [str(x) for x in documentP]
    return flag, documentP, score



def get_word_rank_score(wordR):
    #score = (1+lg(tf))*lg(5002/df) (N=5002)
    global word_list
    global worddf
    global Word_Documentno
    global dictionary
    
    scoreR = []
    log = math.log
    
    word_document_number = Word_Documentno[word_list.index(wordR)]
    Df = float(worddf[word_list.index(wordR)])
    for i in word_document_number:   
        Tf = len([y for y,x in enumerate(dictionary[i]) if x == wordR])
        Score = (1+log(Tf,10))*log((5002.0/Df),10)
        Score = format(Score, '.4f')
        Score = float(Score)
        scoreR.append(Score)
    return word_document_number,scoreR
    

def Rank(query):
    global stop_words
    
    porter_stemmer = PorterStemmer()
    Word_use = []
    Wdocument_number = []
    WScore = []
    Dic_words_score = {}
    Rdocument = []
    Rscore = []
    

    punctuation_string = string.punctuation
    for i in query:
        if i in string.punctuation:
            query = query.replace(i," ")
    
    q = query.split()
    for qu in q:
        if qu not in stop_words:
            qu = porter_stemmer.stem(qu)
            Word_use.append(qu)
    
    for w in Word_use:
        Wdocument_number, WScore = get_word_rank_score(w)
        a = len(Wdocument_number)
        for y in range(0,a):
            if Wdocument_number[y] not in Dic_words_score:
                Dic_words_score[Wdocument_number[y]] = float(WScore[y])
            else:
                Dic_words_score[Wdocument_number[y]] = float(Dic_words_score[Wdocument_number[y]])+float(WScore[y])
    
    Dic_words_score = sorted(Dic_words_score.items(), key=lambda d: d[1], reverse=True)
    for term in Dic_words_score:
        Rdocument.append(term[0])
        Rscore.append(term[1])
        
    flag = 0
    return flag, Rdocument, Rscore


In [38]:
def QueryType(query):
    
    q = re.split(" +(AND NOT|AND|OR) +", query)
    Q=query.split()

    if len(q) > 1:
        word1 = q[0]
        operator = q[1]
        word2 = q[2]
        return BooleanSearch(word1, operator, word2)

    else:
        if '"' in query:
            return PhraseSearch(query, prox=1)
        elif '#' in query:
            word1, word2, proximity_number = get_proxomity_query_elements(query)
            return Proximitysearch(word1, word2, proximity_number)
        else:
            if len(Q) == 1:
                return Search(query)
            else:
                return Rank(query)

                
def excuteQueries(querypath, outputpath):
    
    with open(querypath, 'r') as f:
        queries = f.read().splitlines()
    f.close()

    with open(outputpath, 'w+') as f:
        for line in queries:
            if line[1] != ' ':
                # number of query>=10
                query_number = line[0:2]
                query = line[3:]
            else:
                query_number = line[0]
                query = line[2:]
            
            #query_result = [flag, document id, score]
            #when flag=1, it means query is excuted by Boolean\ Phrase\ Proximity
            #others, query is excuted by Ranked IR
            flag, document, score = QueryType(query)
            if flag == 1:
                for n in document:
                    f.write("{0},{1}\n".format(query_number, n))
            elif flag == 0:
                for i in range(0,150):
                    f.write("{0}, {1}, {2:.4f}\n".format(query_number, document[i],score[i]))
            else:
                f.write("{0}:{1}\n".formate(query_number, 'wrong search'))
    f.close()
    

In [39]:
if __name__ == '__main__':
    
    word_list = []
    stop_words = []
    dictionary = {}
    Word_Documentno = []
    worddf = []
    
    txt_path='D://ATFC//TTDS//Sem1//CW1//CW1collection//trec.5000.xml'
    stop_word_path = 'D://ATFC//TTDS//Sem1//CW1//englishST.txt'
    changeXML(txt_path)
    tokenisation()
    stopwordANDstemming(stop_word_path)
    index()
    
    query_path='D://ATFC//TTDS//Sem1//CW1//CW1collection//queries.ranked.txt'
    outputpath='D://ATFC//TTDS//Sem1//CW1//CW1 up//queries_rank_answer.txt'
    excuteQueries(query_path,outputpath)
    

In [ ]:
if __name__ == '__main__':
    print("*"*80)
    print("\nHello! Here is a clumsy local search engine which 'hope' you are nice day! ")

    txt_path = input("\nPlease enter the path to the Txt you want to use to build the index\nPath: ")
    while not path.exists(collection_path):
            print("\nOhoh...File doesn't exist, please try again!\n")
            txt_path = ("\nPlease enter the path to the Txt you want to use to build the index\nPath: ")

    print("\nInitializing...")
    changeXML(txt_path)
    tokenisation()
    wl = stopwordANDstemming()
    print("\nI'm Writing index...")
    index(wl)
    print("\nNow an Invert Index is ready in your root!")

    loop = True
        while loop:
            querypath = input("\nPlease enter the query file path:\nPath: ")
            while not path.exists(querypath):
                 print("\nFile doesn't exist, try again!\n")
                outputpath = input("\nPlease enter the output file path:\nPath: ")
            outputpath = input("\nPlease enter the output file path:\nPath: ")

            print("\nI'm Searching...")
            excuteQueries(querypath, outputpath)
            print("\nThe result is saved in your root!")

            q = input("\n\n\nDo you want to perform another search? [Y]/[N]\n")
            while q.lower() not in ['y', 'n']:
                print("\nOhoh...It's not a valid option, pleace try again!\n")
                q = input("\nDo you want to perform another search? [Y]/[N]\n")
            if q.lower() == 'y':
                loop = True
            elif q.lower() == 'n':
                print("*" * 80)
                print("\t\t\tThank you very much! Again~Have a nice day!(*^▽^*)")
                print("*" * 80)
                loop = False

In [20]:
Score = 2.365229842
Score = format(Score, '.4f')
Score = float(Score)
if isinstance(Score,str) == True:
    print('str')
elif isinstance(Score,float) == True:
    print('float')
else:
    print('other')

float
